# Metody ewolucyjne i uczenie się maszyn
## Na kilku zadaniach klasyfikacji z repozytorium UCI porównać metody xgboost i catboost
### Agnieszka Czaplicka, Bartosz Sowul

# test: instalacja potrzebnych modułów

In [3]:
!pip install catboost
!pip install xgboost
!pip install matplotlib
!pip install pandas==0.20.3
!pip install numpy
!pip install scikit_learn
!pip install xlrd

## Ładowanie potrzebnych modułów

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold

## Funkcje pomocnicze

### Przygotowanie zbiorów

In [5]:
# zbiory Adult/Annealing/Breast/CTG

from sklearn.preprocessing import MinMaxScaler, Imputer

def prepare(filename):
    if filename[-7:] == 'CTG.xls':
        df = pd.read_excel(filename, sheetname='Data', header=None, parse_cols='K:AE,AT', skiprows=2, nrows=2126)
        print(df.tail())
        df = df[:-3]
        print(df.tail())
    else:
        df = pd.read_csv(filename, header=None)
    
    if filename[-32:] == 'breast-cancer-wisconsin.data.txt':
        df[6] = df[6].replace('?', 0)
        df[6] = df[6].astype('int', errors='ignore')
        df[6] = df[6].replace(0, np.median(df[6]))
    #print(df.dtypes)

    array = df.values
    X = array[:, 0:df.shape[1]-1]
    print('Przetwarzanie wstępne...')
    for i in range(np.shape(X)[1]):
        if df[i].dtype == object:
            names_in_col = df[i].unique()
            names_in_col = sorted(names_in_col, key=lambda v: (v.lower(), v))
            if names_in_col[0].strip() == '?':
                names_in_col = names_in_col[1:]
            elif names_in_col[len(names_in_col) - 1].strip() == '?':
                names_in_col = names_in_col[:-1]
            #print(names_in_col)
            col_dict = dict(zip(names_in_col, range(1, len(names_in_col)+1)))
            X[:, i] = np.array([col_dict.get(elem, np.NaN) for elem in df[i]])
        else:
            df[i] = df[i].replace('?', np.NaN)
            # df[i] = df[i].fillna(value=0)
            X[:, i] = np.array(df[i])

    if df[df.shape[1]-1].dtype == object:
        names_in_col = df[df.shape[1]-1].unique()
        names_in_col = sorted(names_in_col, key=lambda v: (v.lower(), v))
        #print(names_in_col)
        col_dict = dict(zip(names_in_col, range(len(names_in_col))))
        y = np.array([col_dict.get(elem) for elem in df[df.shape[1]-1]])
    else:
        y = array[:, df.shape[1]-1]

    imputer = Imputer(missing_values='NaN', strategy='median', axis=0)
    X = imputer.fit_transform(X)
    # scaler = MinMaxScaler(feature_range=(0, 1)) # skalowanie przy walidacji
    # X = scaler.fit_transform(X)
    print('Przetworzono.')
    return X, y


print("Adult")
X, y = prepare('./data/adult/adult.data.txt')
print("\n------------------------------------------------\n\nAnnealing")
X2, y2 = prepare('./data/annealing/anneal.data.txt')
X2 = X2[:, np.concatenate((np.arange(5), [8, 26, 27, 28, 29, 31]), axis=None)]
print("\n------------------------------------------------\n\nBreast")
X3, y3 = prepare('./data/breast/breast-cancer-wisconsin.data.txt')
X3 = X3[:, 1:]
print("\n------------------------------------------------\n\nCTG")
X4, y4 = prepare('./data/cardiotocography/CTG.xls')


Adult
Przetwarzanie wstępne...
Przetworzono.

------------------------------------------------

Annealing
Przetwarzanie wstępne...
Przetworzono.

------------------------------------------------

Breast
Przetwarzanie wstępne...
Przetworzono.

------------------------------------------------

CTG
         0         1         2         3         4         5         6     7   \
2124  140.0  0.000679  0.000000  0.006110  0.000000  0.000000  0.000000  78.0   
2125  142.0  0.001616  0.001616  0.008078  0.000000  0.000000  0.000000  74.0   
2126    NaN       NaN       NaN       NaN       NaN       NaN       NaN   NaN   
2127    NaN       NaN       NaN       NaN  0.000000  0.000000  0.000000   NaN   
2128    NaN       NaN  0.480634  0.014925  0.015385  0.001353  0.005348  87.0   

       8     9  ...      12     13   14   15     16     17     18   19   20  \
2124  0.4  27.0 ...   103.0  169.0  6.0  0.0  152.0  147.0  151.0  4.0  1.0   
2125  0.4  36.0 ...   117.0  159.0  2.0  1.0  145.0  143.0

### K-krotna walidacja krzyżowa

In [6]:
def cross_validation(X, y, K, clf):
    n = np.shape(X)[0]
    K_cv_ind = np.random.permutation(n)
    for k in range(K):
        print('\nk = ', k)
        ind_test = K_cv_ind[round(k * (1 / K) * n): round((k+1) * (1 / K) * n)]
        #print('test:', ind_test)
        X_test = X[ind_test, :]
        y_test = y[ind_test]
        if k != 0:
            ind_train = K_cv_ind[np.concatenate((np.arange(round(k * (1 / K) * n)),
                                                 np.arange(round((k+1) * (1 / K) * n), n)), axis=None)]
            X_train = X[ind_train, :]
            y_train = y[ind_train]
        else:
            ind_train = K_cv_ind[range(round((k+1) * (1 / K) * n), n)]
            X_train = X[ind_train, :]
            y_train = y[ind_train]
        #print('train:', ind_train)

        max_train = np.amax(X_train, axis=0)
        min_train = np.amin(X_train, axis=0)
        X_train = 2 * (X_train - min_train) / (max_train - min_train) - 1
        X_test = 2 * (X_test - min_train) / (max_train - min_train) - 1

        find_params(clf, X_train, y_train)
        #params = find_params(clf, X_train, y_train)
        #values = predict(X_test, y_test)
        
#cross_validation(X, y, 5, clf1)

### Optymalizacja hiperparametrów przy użyciu RandomizedSearchCV

### Funkcje pomocnicze do raportowania i zapisu wyników eksperymentów

In [7]:
import csv
from pathlib import Path

def save_results(results, filename='results.csv'):
    mode = 'w'
    path = Path(filename)
    if path.is_file():
         mode = 'a'
    with open(filename, mode) as f, open('params.csv', 'a') as f2:
        w = csv.writer(f)
        w2 = csv.writer(f2)
        if mode == 'w':
            w.writerow(results[0].keys())
        else:
            pass
        for result in results:
            w.writerow(result.values())
            w2.writerow(result.values())
            w2.writerow(result['params'].keys())
            w2.writerow(result['params'].values())

def report(optimizer, algo, scoring, seed, dataset, n_top=3):
    print('BEST')
    print(f"Mean validation score: {optimizer.cv_results_['mean_test_'+scoring[-1]][optimizer.best_index_]:.3f} \
          (std: {optimizer.cv_results_['std_test_'+scoring[-1]][optimizer.best_index_]:.3f})")
    print(f'Params: {optimizer.best_params_}')
    print()
    
    results = []

    for metric in scoring:
        print(f'Najlepsze hiperparametry ze względu na metrykę {metric}:')
        cv_results = [tup[0] for tup in sorted(enumerate(optimizer.cv_results_['mean_test_'+metric]), key=lambda x:x[1])]
        for idx in cv_results[::-1][:3]:
            results.append({'dataset': dataset, 'seed': seed, 'algo': type(algo).__name__,
                            'metric': metric, 'model_id': idx,
                            'mean_test_': optimizer.cv_results_['mean_test_'+metric][idx],
                            'std_test_': optimizer.cv_results_['std_test_'+metric][idx],
                            'params': optimizer.cv_results_['params'][idx]})
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                      optimizer.cv_results_['mean_test_'+metric][idx],
                      optimizer.cv_results_['std_test_'+metric][idx]))
            print("Params: {0}".format(optimizer.cv_results_['params'][idx]))
            print()
    else:
        save_results(results)

In [8]:
#accuracy, F1 score i logloss
from time import time

from catboost import CatBoostClassifier
import scipy.stats as st
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier

def find_params(clf, scoring, seed, X=None, y=None):
    params = None
    if type(clf) == XGBClassifier:
        print(f'Optymalizacja hiperparametrów {type(clf).__name__}...')
#learning_rate
#gamma
#max_depth
#min_child_weight
#subsample
#colsample_bytree
#reg_alpha
#reg_lambda
#n_estimators
        params = {
            'learning_rate': st.uniform(0.05, 1.0),
            'gamma': st.uniform(0, 10),
            'max_depth': st.randint(3, 17),
            'min_child_weight': st.expon(0, 50),
            'subsample': st.beta(10, 1),
            'colsample_bytree': st.beta(10, 1) ,
            'reg_alpha': st.expon(0, 50),
            'reg_lambda': st.randint(0, 100),
            'n_estimators': st.randint(3, 50),
        }
    elif type(clf) == CatBoostClassifier:
#learning_rate
#depth
#l2_leaf_reg
#rsm
#random_strength
#iterations
        print(f'Optymalizacja hiperparametrów {type(clf).__name__}...')
        params = {
            'learning_rate': st.uniform(0.05, 1.0),
            'depth': st.randint(3, 17),
            'l2_leaf_reg': st.randint(0, 100),
            'rsm': st.uniform(0.0, 1.0),
            'random_strength': st.uniform(0.05, 10),
            'iterations': st.randint(3, 50),
        }
    else:
        print(f'Błąd: {type(clf)} klasyfikator nie jest obsługiwany.')
    
    # run randomized search
    n_iter_search = 10
    random_search = RandomizedSearchCV(clf, param_distributions=params,
                                       n_iter=n_iter_search, cv=StratifiedKFold(5), scoring=scoring,
                                       refit='neg_log_loss', return_train_score=True, random_state=seed)

    start = time()
    random_search.fit(X, y)
    print(f'\nRandomizedSearchCV trwało {time() - start:.2f} sekund dla {n_iter_search} kandydatów.')
    return random_search

## Ustawienie parametrów testów

In [9]:
scoring = ['accuracy', 'f1_weighted', 'neg_log_loss']
seeds = [111*n for n in range(0, 15)]
datasets = ['Annealing', 'Breast', 'CTG', 'Adult']

X2, y2 = prepare('./data/annealing/anneal.data.txt')
X3, y3 = prepare('./data/breast/breast-cancer-wisconsin.data.txt')
X4, y4 = prepare('./data/cardiotocography/CTG.xls')
X1, y1 = prepare('./data/adult/adult.data.txt')

XX = [X2, X3, X4, X1]
yy = [y2, y3, y4, y1]

Przetwarzanie wstępne...
Przetworzono.
Przetwarzanie wstępne...
Przetworzono.
         0         1         2         3         4         5         6     7   \
2124  140.0  0.000679  0.000000  0.006110  0.000000  0.000000  0.000000  78.0   
2125  142.0  0.001616  0.001616  0.008078  0.000000  0.000000  0.000000  74.0   
2126    NaN       NaN       NaN       NaN       NaN       NaN       NaN   NaN   
2127    NaN       NaN       NaN       NaN  0.000000  0.000000  0.000000   NaN   
2128    NaN       NaN  0.480634  0.014925  0.015385  0.001353  0.005348  87.0   

       8     9  ...      12     13   14   15     16     17     18   19   20  \
2124  0.4  27.0 ...   103.0  169.0  6.0  0.0  152.0  147.0  151.0  4.0  1.0   
2125  0.4  36.0 ...   117.0  159.0  2.0  1.0  145.0  143.0  145.0  1.0  0.0   
2126  NaN   NaN ...     NaN    NaN  NaN  NaN    NaN    NaN    NaN  NaN  NaN   
2127  NaN   NaN ...     NaN    NaN  NaN  NaN    NaN    NaN    NaN  NaN  NaN   
2128  7.0  91.0 ...     NaN    NaN  NaN 

## TESTY

In [10]:
for seed in seeds:
    for X, y, dataset in zip(XX, yy, datasets):
        print(f'Test zbioru {dataset} przy seedzie {seed}\n')
        xgb = XGBClassifier()
        ctb = CatBoostClassifier(loss_function='MultiClass', verbose=False)

        optimizer_xgb = find_params(xgb, scoring, seed, X, y)
        report(optimizer_xgb, xgb, scoring, seed, dataset)

        optimizer_ctb = find_params(ctb, scoring, seed, X, y)
        report(optimizer_ctb, ctb, scoring, seed, dataset)

Test zbioru Annealing przy seedzie 0

Optymalizacja hiperparametrów XGBClassifier...


/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 


RandomizedSearchCV trwało 4.09 sekund dla 10 kandydatów.
BEST
Mean validation score: -0.675           (std: 0.025)
Params: {'colsample_bytree': 0.94203632548882865, 'gamma': 0.19987665408758737, 'learning_rate': 0.49171092124884536, 'max_depth': 14, 'min_child_weight': 6.4028380737264925, 'n_estimators': 26, 'reg_alpha': 32.782313697880426, 'reg_lambda': 98, 'subsample': 0.84191406829370874}

Najlepsze hiperparametry ze względu na metrykę accuracy:
Mean validation score: 0.762 (std: 0.005)
Params: {'colsample_bytree': 0.85023973979370426, 'gamma': 6.8005556946341228, 'learning_rate': 0.49919773821837204, 'max_depth': 6, 'min_child_weight': 60.111928002778761, 'n_estimators': 25, 'reg_alpha': 84.046110238441372, 'reg_lambda': 75, 'subsample': 0.9116104529398813}

Mean validation score: 0.762 (std: 0.005)
Params: {'colsample_bytree': 0.99718837648301295, 'gamma': 8.2894002921736316, 'learning_rate': 0.054695476192547068, 'max_depth': 16, 'min_child_weight': 42.154151787034785, 'n_estima

/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



RandomizedSearchCV trwało 23.81 sekund dla 10 kandydatów.
BEST
Mean validation score: -0.280           (std: 0.042)
Params: {'depth': 12, 'iterations': 38, 'l2_leaf_reg': 11, 'learning_rate': 0.26038256107384089, 'random_strength': 1.3392629765485331, 'rsm': 0.31542835092418386}

Najlepsze hiperparametry ze względu na metrykę accuracy:
Mean validation score: 0.911 (std: 0.025)
Params: {'depth': 12, 'iterations': 38, 'l2_leaf_reg': 11, 'learning_rate': 0.26038256107384089, 'random_strength': 1.3392629765485331, 'rsm': 0.31542835092418386}

Mean validation score: 0.888 (std: 0.014)
Params: {'depth': 7, 'iterations': 44, 'l2_leaf_reg': 42, 'learning_rate': 0.409507900573786, 'random_strength': 4.4203195379934144, 'rsm': 0.69763119592726486}

Mean validation score: 0.888 (std: 0.018)
Params: {'depth': 9, 'iterations': 42, 'l2_leaf_reg': 87, 'learning_rate': 0.52997717237505737, 'random_strength': 3.9778479610082971, 'rsm': 0.83607876353737753}

Najlepsze hiperparametry ze względu na metry

...


RandomizedSearchCV trwało 1.07 sekund dla 10 kandydatów.
BEST
Mean validation score: -0.273           (std: 0.022)
Params: {'colsample_bytree': 0.94203632548882865, 'gamma': 0.19987665408758737, 'learning_rate': 0.49171092124884536, 'max_depth': 14, 'min_child_weight': 6.4028380737264925, 'n_estimators': 26, 'reg_alpha': 32.782313697880426, 'reg_lambda': 98, 'subsample': 0.84191406829370874}

Najlepsze hiperparametry ze względu na metrykę accuracy:
Mean validation score: 0.941 (std: 0.021)
Params: {'colsample_bytree': 0.94203632548882865, 'gamma': 0.19987665408758737, 'learning_rate': 0.49171092124884536, 'max_depth': 14, 'min_child_weight': 6.4028380737264925, 'n_estimators': 26, 'reg_alpha': 32.782313697880426, 'reg_lambda': 98, 'subsample': 0.84191406829370874}

Mean validation score: 0.914 (std: 0.020)
Params: {'colsample_bytree': 0.99718837648301295, 'gamma': 8.2894002921736316, 'learning_rate': 0.054695476192547068, 'max_depth': 16, 'min_child_weight': 42.154151787034785, 'n_est

/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 


RandomizedSearchCV trwało 5.61 sekund dla 10 kandydatów.
BEST
Mean validation score: -0.397           (std: 0.152)
Params: {'colsample_bytree': 0.94203632548882865, 'gamma': 0.19987665408758737, 'learning_rate': 0.49171092124884536, 'max_depth': 14, 'min_child_weight': 6.4028380737264925, 'n_estimators': 26, 'reg_alpha': 32.782313697880426, 'reg_lambda': 98, 'subsample': 0.84191406829370874}

Najlepsze hiperparametry ze względu na metrykę accuracy:
Mean validation score: 0.866 (std: 0.058)
Params: {'colsample_bytree': 0.94203632548882865, 'gamma': 0.19987665408758737, 'learning_rate': 0.49171092124884536, 'max_depth': 14, 'min_child_weight': 6.4028380737264925, 'n_estimators': 26, 'reg_alpha': 32.782313697880426, 'reg_lambda': 98, 'subsample': 0.84191406829370874}

Mean validation score: 0.828 (std: 0.109)
Params: {'colsample_bytree': 0.87970882158714636, 'gamma': 1.381829513486138, 'learning_rate': 0.24658236168005349, 'max_depth': 3, 'min_child_weight': 54.313468880808479, 'n_estima

/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)

Iteration with suspicious time 8.47 sec ignored in overall statistics.

Iteration with suspicious time 10.2 sec ignored in overall statistics.

Iteration with suspicious time 9.95 sec ignored in overall statistics.

Iteration with suspicious time 9.09 sec ignored in overall statistics.

Iteration with suspicious time 9.08 sec ignored in overall statistics.

Iteration with suspicious time 12.1 sec ignored in overall statistics.


KeyboardInterrupt: 

## Testy na zbiorze Annealing

In [12]:
print("Zbiór Annealing")

scoring = ['accuracy', 'f1_weighted', 'neg_log_loss']

dataset = 'Annealing'
X, y = prepare('./data/annealing/anneal.data.txt')

for seed in seeds:
    print(f'Test zbioru {dataset} przy seedzie {seed}\n')
    xgb = XGBClassifier()
    ctb = CatBoostClassifier(loss_function='MultiClass', verbose=False)
    
    optimizer_xgb2 = find_params(xgb, scoring, seed, X, y)
    report(optimizer_xgb2, xgb, scoring, seed, dataset)

    optimizer_ctb2 = find_params(ctb, scoring, seed, X, y)
    report(optimizer_ctb2, ctb, scoring, seed, dataset)

Zbiór Annealing
Przetwarzanie wstępne...
Przetworzono.
Test zbioru Annealing przy seedzie 0

Optymalizacja hiperparametrów XGBClassifier...


/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 


RandomizedSearchCV trwało 4.12 sekund dla 10 kandydatów.
BEST
Mean validation score: -0.675           (std: 0.025)
Params: {'colsample_bytree': 0.94203632548882865, 'gamma': 0.19987665408758737, 'learning_rate': 0.49171092124884536, 'max_depth': 14, 'min_child_weight': 6.4028380737264925, 'n_estimators': 26, 'reg_alpha': 32.782313697880426, 'reg_lambda': 98, 'subsample': 0.84191406829370874}

Najlepsze hiperparametry ze względu na metrykę accuracy:
Mean validation score: 0.762 (std: 0.005)
Params: {'colsample_bytree': 0.85023973979370426, 'gamma': 6.8005556946341228, 'learning_rate': 0.49919773821837204, 'max_depth': 6, 'min_child_weight': 60.111928002778761, 'n_estimators': 25, 'reg_alpha': 84.046110238441372, 'reg_lambda': 75, 'subsample': 0.9116104529398813}

Mean validation score: 0.762 (std: 0.005)
Params: {'colsample_bytree': 0.99718837648301295, 'gamma': 8.2894002921736316, 'learning_rate': 0.054695476192547068, 'max_depth': 16, 'min_child_weight': 42.154151787034785, 'n_estima

/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



RandomizedSearchCV trwało 28.22 sekund dla 10 kandydatów.
BEST
Mean validation score: -0.280           (std: 0.042)
Params: {'depth': 12, 'iterations': 38, 'l2_leaf_reg': 11, 'learning_rate': 0.26038256107384089, 'random_strength': 1.3392629765485331, 'rsm': 0.31542835092418386}

Najlepsze hiperparametry ze względu na metrykę accuracy:
Mean validation score: 0.911 (std: 0.025)
Params: {'depth': 12, 'iterations': 38, 'l2_leaf_reg': 11, 'learning_rate': 0.26038256107384089, 'random_strength': 1.3392629765485331, 'rsm': 0.31542835092418386}

Mean validation score: 0.888 (std: 0.014)
Params: {'depth': 7, 'iterations': 44, 'l2_leaf_reg': 42, 'learning_rate': 0.409507900573786, 'random_strength': 4.4203195379934144, 'rsm': 0.69763119592726486}

Mean validation score: 0.888 (std: 0.018)
Params: {'depth': 9, 'iterations': 42, 'l2_leaf_reg': 87, 'learning_rate': 0.52997717237505737, 'random_strength': 3.9778479610082971, 'rsm': 0.83607876353737753}

Najlepsze hiperparametry ze względu na metry

...


RandomizedSearchCV trwało 260.95 sekund dla 10 kandydatów.
BEST
Mean validation score: -0.178           (std: 0.088)
Params: {'depth': 9, 'iterations': 48, 'l2_leaf_reg': 3, 'learning_rate': 1.0337639362904552, 'random_strength': 3.6010401334360274, 'rsm': 0.12218409302241851}

Najlepsze hiperparametry ze względu na metrykę accuracy:
Mean validation score: 0.944 (std: 0.014)
Params: {'depth': 9, 'iterations': 48, 'l2_leaf_reg': 3, 'learning_rate': 1.0337639362904552, 'random_strength': 3.6010401334360274, 'rsm': 0.12218409302241851}

Mean validation score: 0.942 (std: 0.027)
Params: {'depth': 8, 'iterations': 48, 'l2_leaf_reg': 13, 'learning_rate': 0.59625009404000295, 'random_strength': 6.0256008230431997, 'rsm': 0.99851864875211649}

Mean validation score: 0.916 (std: 0.013)
Params: {'depth': 14, 'iterations': 47, 'l2_leaf_reg': 84, 'learning_rate': 0.50120274886919458, 'random_strength': 0.67432491495970526, 'rsm': 0.79281904279625304}

Najlepsze hiperparametry ze względu na metryk

/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 


RandomizedSearchCV trwało 3.68 sekund dla 10 kandydatów.
BEST
Mean validation score: -0.702           (std: 0.034)
Params: {'colsample_bytree': 0.95155332065877152, 'gamma': 0.11587131064204281, 'learning_rate': 0.55127938122201181, 'max_depth': 3, 'min_child_weight': 12.090963911948858, 'n_estimators': 47, 'reg_alpha': 26.712336537794869, 'reg_lambda': 94, 'subsample': 0.77350992433529686}

Najlepsze hiperparametry ze względu na metrykę accuracy:
Mean validation score: 0.762 (std: 0.005)
Params: {'colsample_bytree': 0.90861507957508103, 'gamma': 7.5524297277391685, 'learning_rate': 0.095199585423121305, 'max_depth': 11, 'min_child_weight': 151.24403238792434, 'n_estimators': 13, 'reg_alpha': 21.440227167107949, 'reg_lambda': 92, 'subsample': 0.73546471281372006}

Mean validation score: 0.762 (std: 0.005)
Params: {'colsample_bytree': 0.97546094481662693, 'gamma': 6.532811596262194, 'learning_rate': 0.19661149145163986, 'max_depth': 9, 'min_child_weight': 23.741926727042287, 'n_estimat

/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



RandomizedSearchCV trwało 171.73 sekund dla 10 kandydatów.
BEST
Mean validation score: -0.188           (std: 0.059)
Params: {'depth': 13, 'iterations': 48, 'l2_leaf_reg': 27, 'learning_rate': 0.99618131953081424, 'random_strength': 7.7434159854189426, 'rsm': 0.43478642701713144}

Najlepsze hiperparametry ze względu na metrykę accuracy:
Mean validation score: 0.946 (std: 0.021)
Params: {'depth': 13, 'iterations': 48, 'l2_leaf_reg': 27, 'learning_rate': 0.99618131953081424, 'random_strength': 7.7434159854189426, 'rsm': 0.43478642701713144}

Mean validation score: 0.915 (std: 0.018)
Params: {'depth': 16, 'iterations': 39, 'l2_leaf_reg': 28, 'learning_rate': 0.63886259261387701, 'random_strength': 6.1497616996517896, 'rsm': 0.20095988066638659}

Mean validation score: 0.866 (std: 0.029)
Params: {'depth': 15, 'iterations': 29, 'l2_leaf_reg': 68, 'learning_rate': 0.64615044772838581, 'random_strength': 9.1517464776580368, 'rsm': 0.4450405843072055}

Najlepsze hiperparametry ze względu na m

## Testy na zbiorze Breast

In [14]:
print("Zbiór Breast")

scoring = ['accuracy', 'f1_weighted', 'neg_log_loss']

dataset = 'Breast'
X, y = prepare('./data/breast/breast-cancer-wisconsin.data.txt')

for seed in seeds:
    print(f'Test zbioru {dataset} przy seedzie {seed}\n')
    xgb = XGBClassifier()
    ctb = CatBoostClassifier(loss_function='MultiClass', verbose=False)
    
    optimizer_xgb3 = find_params(xgb, scoring, seed, X, y)
    report(optimizer_xgb3, xgb, scoring, seed, dataset)

    optimizer_ctb3 = find_params(ctb, scoring, seed, X, y)
    report(optimizer_ctb3, ctb, scoring, seed, dataset)
else:
    print('Done.\n')

Zbiór Breast
Przetwarzanie wstępne...
Przetworzono.
Test zbioru Breast przy seedzie 0

Optymalizacja hiperparametrów XGBClassifier...


/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 


RandomizedSearchCV trwało 1.01 sekund dla 10 kandydatów.
BEST
Mean validation score: -0.273           (std: 0.022)
Params: {'colsample_bytree': 0.94203632548882865, 'gamma': 0.19987665408758737, 'learning_rate': 0.49171092124884536, 'max_depth': 14, 'min_child_weight': 6.4028380737264925, 'n_estimators': 26, 'reg_alpha': 32.782313697880426, 'reg_lambda': 98, 'subsample': 0.84191406829370874}

Najlepsze hiperparametry ze względu na metrykę accuracy:
Mean validation score: 0.941 (std: 0.021)
Params: {'colsample_bytree': 0.94203632548882865, 'gamma': 0.19987665408758737, 'learning_rate': 0.49171092124884536, 'max_depth': 14, 'min_child_weight': 6.4028380737264925, 'n_estimators': 26, 'reg_alpha': 32.782313697880426, 'reg_lambda': 98, 'subsample': 0.84191406829370874}

Mean validation score: 0.914 (std: 0.020)
Params: {'colsample_bytree': 0.99718837648301295, 'gamma': 8.2894002921736316, 'learning_rate': 0.054695476192547068, 'max_depth': 16, 'min_child_weight': 42.154151787034785, 'n_est

/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 


RandomizedSearchCV trwało 1.08 sekund dla 10 kandydatów.
BEST
Mean validation score: -0.171           (std: 0.039)
Params: {'colsample_bytree': 0.86900124541044066, 'gamma': 2.1049924581368176, 'learning_rate': 0.85206457718643547, 'max_depth': 3, 'min_child_weight': 8.1923283466419132, 'n_estimators': 39, 'reg_alpha': 11.619779341850629, 'reg_lambda': 20, 'subsample': 0.9003944179579163}

Najlepsze hiperparametry ze względu na metrykę accuracy:
Mean validation score: 0.957 (std: 0.025)
Params: {'colsample_bytree': 0.86900124541044066, 'gamma': 2.1049924581368176, 'learning_rate': 0.85206457718643547, 'max_depth': 3, 'min_child_weight': 8.1923283466419132, 'n_estimators': 39, 'reg_alpha': 11.619779341850629, 'reg_lambda': 20, 'subsample': 0.9003944179579163}

Mean validation score: 0.944 (std: 0.025)
Params: {'colsample_bytree': 0.96822823759893117, 'gamma': 3.6077620064287794, 'learning_rate': 0.8997204066587916, 'max_depth': 11, 'min_child_weight': 19.197911834329673, 'n_estimators'

...


RandomizedSearchCV trwało 95.23 sekund dla 10 kandydatów.
BEST
Mean validation score: -0.110           (std: 0.048)
Params: {'depth': 3, 'iterations': 22, 'l2_leaf_reg': 25, 'learning_rate': 0.46071028842540224, 'random_strength': 2.7700487385172354, 'rsm': 0.31277368098313874}

Najlepsze hiperparametry ze względu na metrykę accuracy:
Mean validation score: 0.967 (std: 0.018)
Params: {'depth': 16, 'iterations': 32, 'l2_leaf_reg': 78, 'learning_rate': 0.44660573759201611, 'random_strength': 4.831289043644305, 'rsm': 0.47379609759977315}

Mean validation score: 0.963 (std: 0.016)
Params: {'depth': 3, 'iterations': 22, 'l2_leaf_reg': 25, 'learning_rate': 0.46071028842540224, 'random_strength': 2.7700487385172354, 'rsm': 0.31277368098313874}

Mean validation score: 0.963 (std: 0.023)
Params: {'depth': 12, 'iterations': 39, 'l2_leaf_reg': 11, 'learning_rate': 0.11213484913016254, 'random_strength': 0.76798594561040323, 'rsm': 0.15559777724403012}

Najlepsze hiperparametry ze względu na met

/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 


RandomizedSearchCV trwało 0.90 sekund dla 10 kandydatów.
BEST
Mean validation score: -0.207           (std: 0.033)
Params: {'colsample_bytree': 0.67703369264183955, 'gamma': 6.4028671316932613, 'learning_rate': 0.35632644478261905, 'max_depth': 6, 'min_child_weight': 24.80536862086241, 'n_estimators': 30, 'reg_alpha': 8.906916079824919, 'reg_lambda': 21, 'subsample': 0.8593258155875817}

Najlepsze hiperparametry ze względu na metrykę accuracy:
Mean validation score: 0.946 (std: 0.022)
Params: {'colsample_bytree': 0.67703369264183955, 'gamma': 6.4028671316932613, 'learning_rate': 0.35632644478261905, 'max_depth': 6, 'min_child_weight': 24.80536862086241, 'n_estimators': 30, 'reg_alpha': 8.906916079824919, 'reg_lambda': 21, 'subsample': 0.8593258155875817}

Mean validation score: 0.940 (std: 0.033)
Params: {'colsample_bytree': 0.95155332065877152, 'gamma': 0.11587131064204281, 'learning_rate': 0.55127938122201181, 'max_depth': 3, 'min_child_weight': 12.090963911948858, 'n_estimators': 4


Iteration with suspicious time 0.646 sec ignored in overall statistics.

Iteration with suspicious time 0.631 sec ignored in overall statistics.

Iteration with suspicious time 0.667 sec ignored in overall statistics.



RandomizedSearchCV trwało 60.60 sekund dla 10 kandydatów.
BEST
Mean validation score: -0.117           (std: 0.054)
Params: {'depth': 13, 'iterations': 48, 'l2_leaf_reg': 27, 'learning_rate': 0.99618131953081424, 'random_strength': 7.7434159854189426, 'rsm': 0.43478642701713144}

Najlepsze hiperparametry ze względu na metrykę accuracy:
Mean validation score: 0.969 (std: 0.021)
Params: {'depth': 6, 'iterations': 37, 'l2_leaf_reg': 99, 'learning_rate': 0.29029261336749473, 'random_strength': 6.4083060702273746, 'rsm': 0.37355856982970947}

Mean validation score: 0.961 (std: 0.013)
Params: {'depth': 10, 'iterations': 12, 'l2_leaf_reg': 21, 'learning_rate': 0.91027965019544976, 'random_strength': 7.3615696497664223, 'rsm': 0.72950419213897844}

Mean validation score: 0.960 (std: 0.020)
Params: {'depth': 13, 'iterations': 48, 'l2_leaf_reg': 27, 'learning_rate': 0.99618131953081424, 'random_strength': 7.7434159854189426, 'rsm': 0.43478642701713144}

Najlepsze hiperparametry ze względu na me

## Testy na zbiorze CTG

In [ ]:
print("Zbiór CTG")

scoring = ['accuracy', 'f1_weighted', 'neg_log_loss']

dataset = 'CTG'
X, y = prepare('./data/cardiotocography/CTG.xls')

for seed in seeds:
    print(f'Test zbioru {dataset} przy seedzie {seed}\n')
    xgb = XGBClassifier()
    ctb = CatBoostClassifier(loss_function='MultiClass', verbose=False)
    
    optimizer_xgb4 = find_params(xgb, scoring, seed, X, y)
    report(optimizer_xgb4, xgb, scoring, seed, dataset)

    optimizer_ctb4 = find_params(ctb, scoring, seed, X, y)
    report(optimizer_ctb4, ctb, scoring, seed, dataset)
else:
    print('Done.\n')

## Testy na zbiorze Adult

In [13]:
print("Zbiór Adult")

scoring = ['accuracy', 'f1_weighted', 'neg_log_loss']

dataset = 'Adult'
X, y = prepare('./data/adult/adult.data.txt')

for seed in seeds:
    print(f'Test zbioru {dataset} przy seedzie {seed}\n')
    xgb = XGBClassifier()
    ctb = CatBoostClassifier(loss_function='MultiClass', verbose=False)
    
    optimizer_xgb1 = find_params(xgb, scoring, seed, X, y)
    report(optimizer_xgb1, xgb, scoring, seed, dataset)

    optimizer_ctb1 = find_params(ctb, scoring, seed, X, y)
    report(optimizer_ctb1, ctb, scoring, seed, dataset)
else:
    print('Done.\n')

Zbiór Adult
Przetwarzanie wstępne...
Przetworzono.
Test zbioru Adult przy seedzie 0

Optymalizacja hiperparametrów XGBClassifier...

RandomizedSearchCV trwało 50.93 sekund dla 10 kandydatów.
BEST
Mean validation score: -0.302           (std: 0.005)
Params: {'colsample_bytree': 0.94203632548882865, 'gamma': 0.19987665408758737, 'learning_rate': 0.49171092124884536, 'max_depth': 14, 'min_child_weight': 6.4028380737264925, 'n_estimators': 26, 'reg_alpha': 32.782313697880426, 'reg_lambda': 98, 'subsample': 0.84191406829370874}

Najlepsze hiperparametry ze względu na metrykę accuracy:
Mean validation score: 0.865 (std: 0.004)
Params: {'colsample_bytree': 0.94203632548882865, 'gamma': 0.19987665408758737, 'learning_rate': 0.49171092124884536, 'max_depth': 14, 'min_child_weight': 6.4028380737264925, 'n_estimators': 26, 'reg_alpha': 32.782313697880426, 'reg_lambda': 98, 'subsample': 0.84191406829370874}

Mean validation score: 0.860 (std: 0.004)
Params: {'colsample_bytree': 0.83870821640103754

learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease l


RandomizedSearchCV trwało 319.70 sekund dla 10 kandydatów.
BEST
Mean validation score: -0.291           (std: 0.004)
Params: {'depth': 5, 'iterations': 46, 'l2_leaf_reg': 86, 'learning_rate': 0.84396256047962848, 'random_strength': 8.4556964872428129, 'rsm': 0.81520745745294065}

Najlepsze hiperparametry ze względu na metrykę accuracy:
Mean validation score: 0.867 (std: 0.005)
Params: {'depth': 5, 'iterations': 46, 'l2_leaf_reg': 86, 'learning_rate': 0.84396256047962848, 'random_strength': 8.4556964872428129, 'rsm': 0.81520745745294065}

Mean validation score: 0.865 (std: 0.004)
Params: {'depth': 6, 'iterations': 36, 'l2_leaf_reg': 38, 'learning_rate': 0.89380186955457119, 'random_strength': 7.3120188758759106, 'rsm': 0.27030569335406984}

Mean validation score: 0.859 (std: 0.003)
Params: {'depth': 7, 'iterations': 47, 'l2_leaf_reg': 84, 'learning_rate': 0.35178945091410735, 'random_strength': 8.5890645838724424, 'rsm': 0.29771597903464209}

Najlepsze hiperparametry ze względu na metr


Iteration with suspicious time 10.2 sec ignored in overall statistics.

Iteration with suspicious time 10.3 sec ignored in overall statistics.

Iteration with suspicious time 10.8 sec ignored in overall statistics.



RandomizedSearchCV trwało 950.20 sekund dla 10 kandydatów.
BEST
Mean validation score: -0.295           (std: 0.004)
Params: {'depth': 12, 'iterations': 30, 'l2_leaf_reg': 4, 'learning_rate': 0.40915509470067718, 'random_strength': 4.7258903421436473, 'rsm': 0.46122784046875787}

Najlepsze hiperparametry ze względu na metrykę accuracy:
Mean validation score: 0.864 (std: 0.003)
Params: {'depth': 12, 'iterations': 30, 'l2_leaf_reg': 4, 'learning_rate': 0.40915509470067718, 'random_strength': 4.7258903421436473, 'rsm': 0.46122784046875787}

Mean validation score: 0.861 (std: 0.004)
Params: {'depth': 16, 'iterations': 46, 'l2_leaf_reg': 8, 'learning_rate': 0.14621393483232753, 'random_strength': 2.1324967538930815, 'rsm': 0.57654317269310351}

Mean validation score: 0.855 (std: 0.005)
Params: {'depth': 9, 'iterations': 7, 'l2_leaf_reg': 98, 'learning_rate': 0.91904354541926259, 'random_strength': 1.3995625029681635, 'rsm': 0.582102452009569}

Najlepsze hiperparametry ze względu na metrykę

...


RandomizedSearchCV trwało 240.91 sekund dla 10 kandydatów.
BEST
Mean validation score: -0.289           (std: 0.004)
Params: {'depth': 11, 'iterations': 42, 'l2_leaf_reg': 27, 'learning_rate': 0.46235778831205748, 'random_strength': 2.8663896255962236, 'rsm': 0.3902777805896922}

Najlepsze hiperparametry ze względu na metrykę accuracy:
Mean validation score: 0.868 (std: 0.004)
Params: {'depth': 13, 'iterations': 46, 'l2_leaf_reg': 78, 'learning_rate': 0.45514183627531674, 'random_strength': 8.72968707036061, 'rsm': 0.50328774703970058}

Mean validation score: 0.867 (std: 0.003)
Params: {'depth': 11, 'iterations': 42, 'l2_leaf_reg': 27, 'learning_rate': 0.46235778831205748, 'random_strength': 2.8663896255962236, 'rsm': 0.3902777805896922}

Mean validation score: 0.866 (std: 0.004)
Params: {'depth': 3, 'iterations': 48, 'l2_leaf_reg': 8, 'learning_rate': 0.6043858124306527, 'random_strength': 6.331265224775608, 'rsm': 0.69739294419434827}

Najlepsze hiperparametry ze względu na metrykę 

learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease l


RandomizedSearchCV trwało 1001.55 sekund dla 10 kandydatów.
BEST
Mean validation score: -0.283           (std: 0.004)
Params: {'depth': 8, 'iterations': 48, 'l2_leaf_reg': 13, 'learning_rate': 0.59625009404000295, 'random_strength': 6.0256008230431997, 'rsm': 0.99851864875211649}

Najlepsze hiperparametry ze względu na metrykę accuracy:
Mean validation score: 0.871 (std: 0.003)
Params: {'depth': 14, 'iterations': 47, 'l2_leaf_reg': 84, 'learning_rate': 0.50120274886919458, 'random_strength': 0.67432491495970526, 'rsm': 0.79281904279625304}

Mean validation score: 0.871 (std: 0.004)
Params: {'depth': 8, 'iterations': 48, 'l2_leaf_reg': 13, 'learning_rate': 0.59625009404000295, 'random_strength': 6.0256008230431997, 'rsm': 0.99851864875211649}

Mean validation score: 0.866 (std: 0.004)
Params: {'depth': 12, 'iterations': 32, 'l2_leaf_reg': 55, 'learning_rate': 0.40217668314456684, 'random_strength': 0.92239791432622509, 'rsm': 0.4219936725567961}

Najlepsze hiperparametry ze względu na 


Iteration with suspicious time 4.87 sec ignored in overall statistics.
/media/bs/data/projects/0_moje_portfolio/eiti/eiti/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



RandomizedSearchCV trwało 758.42 sekund dla 10 kandydatów.
BEST
Mean validation score: -0.287           (std: 0.004)
Params: {'depth': 13, 'iterations': 48, 'l2_leaf_reg': 27, 'learning_rate': 0.99618131953081424, 'random_strength': 7.7434159854189426, 'rsm': 0.43478642701713144}

Najlepsze hiperparametry ze względu na metrykę accuracy:
Mean validation score: 0.869 (std: 0.002)
Params: {'depth': 13, 'iterations': 48, 'l2_leaf_reg': 27, 'learning_rate': 0.99618131953081424, 'random_strength': 7.7434159854189426, 'rsm': 0.43478642701713144}

Mean validation score: 0.866 (std: 0.005)
Params: {'depth': 16, 'iterations': 39, 'l2_leaf_reg': 28, 'learning_rate': 0.63886259261387701, 'random_strength': 6.1497616996517896, 'rsm': 0.20095988066638659}

Mean validation score: 0.866 (std: 0.004)
Params: {'depth': 15, 'iterations': 29, 'l2_leaf_reg': 68, 'learning_rate': 0.64615044772838581, 'random_strength': 9.1517464776580368, 'rsm': 0.4450405843072055}

Najlepsze hiperparametry ze względu na m

## Open In Colab Badge

Anybody can open a copy of any github-hosted notebook within Colab. To make it easier to give people access to live views of GitHub-hosted notebooks,
colab provides a [shields.io](http://shields.io/)-style badge, which appears as follows:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

The markdown for the above badge is the following:

```markdown
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)
```

The HTML equivalent is:

```HTML
<a href="https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
```

Remember to replace the notebook URL in this template with the notebook you want to link to.